In [1]:
import sqlite3
from bs4 import BeautifulSoup
import requests
import telegram

In [7]:
DB_DIR = "./user.db"

login_data_skku = {'user_id': 'gyunghoe', 'user_pw': 'Gyunghoe94!!'}
login_url_skku = "https://job.skku.edu/loginproc.aspx"
cookies_skku = get_web_cookie(login_url_skku, login_data_skku)

parsed_recruitment_skku = []

for i in range(1, 3):
    recruitment_url_skku = "http://job.skku.edu/jobs/all/default.aspx?page="+str(i)
    html_skku = get_web_html(recruitment_url_skku, cookies_skku)
    recruitment_json_skku = parse_web_recruitment_skku(html_skku)
    parsed_recruitment_skku+=parse_new_recruitment(recruitment_json_skku, DB_DIR)
    
#여기서부터 아예 다시 하기

In [5]:
login_data_jobkorea = {
    'IP_ONOFF': 'Y', 'DB_Name': 'GG', 'Co_DB_Name': 'GI', 'M_ID': 'gyunghoe', 'M_PWD': 'dgh8915',
 }

login_url_jobkorea = "https://www.jobkorea.co.kr/Login/Login.asp"
cookies_jobkorea = get_web_cookie(login_url_jobkorea, login_data_jobkorea)

recruitment_url_jobkorea = "http://www.jobkorea.co.kr/top100/"
html_jobkorea = get_web_html(recruitment_url_jobkorea, cookies_jobkorea)
recruitment_json_jobkorea = parse_web_recruitment_jobkorea(html_jobkorea)

In [8]:
parsed_recruitment_jobkorea = parse_new_recruitment(recruitment_json_jobkorea, DB_DIR)

In [9]:
print(len(parsed_recruitment_jobkorea))

90


In [12]:
parsed_recruitment = parsed_recruitment_skku + parsed_recruitment_jobkorea

In [14]:
TELEGRAM_BOT_TOKEN = '1098512227:AAFXjpC8rPNmyJlG1NmM-_B80Oed99Yu63s'
ADMIN_TELEGRAM_ID = "1292374436"

In [15]:
chat_last_24_hours = get_telegram_chat_of_last_24_hours(TELEGRAM_BOT_TOKEN)

In [16]:
parsed_chat_last_24_hours = parse_new_chat(chat_last_24_hours, './user.db')

In [17]:
print(parsed_chat_last_24_hours)

[]


In [18]:
if(len(parsed_chat_last_24_hours)>0):
    send_telegram_message(TELEGRAM_BOT_TOKEN, ADMIN_TELEGRAM_ID, "=== 메시지가 있습니다 ===")
    for chat in parsed_chat_last_24_hours:
        send_telegram_message(TELEGRAM_BOT_TOKEN, ADMIN_TELEGRAM_ID, chat['content'])

In [19]:
users = query_all_user_in_db(DB_DIR)

In [20]:
for chat in parsed_chat_last_24_hours:
    users.append(chat['user_id'])

In [21]:
print(len(users))

35


In [22]:
parsed_user = parse_user(users, DB_DIR)

In [23]:
# print(parsed_user)
print(len(parsed_user))

26


In [29]:
for i in parsed_user:
    i = str(i)
    if i=="808560264":
        continue
    try:
        for recruitment in parsed_recruitment:
            continue
#             send_telegram_message(TELEGRAM_BOT_TOKEN, ADMIN_TELEGRAM_ID, make_bot_message_form(recruitment))
        if i=="877064775":
            send_telegram_message(TELEGRAM_BOT_TOKEN, i, "소영아 오늘도 좋은 하루 보내구 사랑해♡")
    except Exception as e:
        send_telegram_message(TELEGRAM_BOT_TOKEN, ADMIN_TELEGRAM_ID, i+str(e))

send_telegram_message(TELEGRAM_BOT_TOKEN, ADMIN_TELEGRAM_ID, "프로그램이 정상적으로 작동을 완료하였습니다")


115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
25


In [28]:
for recruitment in parsed_recruitment:
    send_telegram_message(TELEGRAM_BOT_TOKEN, ADMIN_TELEGRAM_ID, make_bot_message_form(recruitment))

In [2]:
'''
Functions related to database.
This file contains functions to search, insert recruitments and user data to database
'''
import sqlite3

def insert_recruitment_to_db(recruitment, db_dir):
    '''
    insert recruitment data to database
    '''
    conn = sqlite3.connect(db_dir)
    db_cursor = conn.cursor()
    insert_query = 'INSERT INTO rec_db (company, name, deadline) VALUES (?, ?, ?)'
    db_cursor.execute(insert_query,
        (recruitment['company'], recruitment['content'], recruitment['deadline']))
    conn.commit()
    conn.close()

def search_recruitment_duplicate(recruitment, db_dir):
    '''
    searches recruitment data in database
    returns true if there exists same recruitment
    '''
    conn = sqlite3.connect(db_dir)
    db_cursor = conn.cursor()
    search_query = "SELECT * FROM rec_db WHERE company= ? AND name= ?"
    duplicate = db_cursor.execute(search_query,
        (str(recruitment['company']), str(recruitment['content']),))
    conn.commit()
    flag = len(duplicate.fetchall())>0
    conn.close()
    return flag

def parse_new_recruitment(recruitments, db_dir):
    '''
    it takes recruitments and returns new recruitments that does not exist in database
    '''
    refined_recruitments = []
    for recruitment in recruitments:
        if search_recruitment_duplicate(recruitment, db_dir) is True:
            continue
        insert_recruitment_to_db(recruitment, db_dir)
        refined_recruitments.append(recruitment)
    return refined_recruitments


def insert_chat_to_db(chat, db_dir):
    '''
    insert chat data to database
    '''
    conn = sqlite3.connect(db_dir)
    db_cursor = conn.cursor()
    insert_query = 'INSERT INTO chat_db (pk, userid, msg) VALUES (?, ?, ?)'
    db_cursor.execute(insert_query,
        (str(chat['chat_id']), str(chat['user_id']), str(chat['content'])))
    conn.commit()
    conn.close()

def search_chat_duplicate(chat, db_dir):
    '''
    searches chat data in database
    returns true if there exists same chat
    '''
    conn = sqlite3.connect(db_dir)
    db_cursor = conn.cursor()
    search_query = "SELECT pk FROM chat_db WHERE pk=(?)"
    duplicate = db_cursor.execute(search_query, (chat['chat_id'],))
    conn.commit()
    flag = len(duplicate.fetchall())>0
    conn.close()
    return flag

def parse_new_chat(chats, db_dir):
    '''
    it takes chats and returns new chats that does not exist in database
    '''
    refined_chats = []
    for chat in chats:
        if search_chat_duplicate(chat, db_dir) is True:
            continue
        insert_chat_to_db(chat, db_dir)
        refined_chats.append(chat)
    return refined_chats

def query_all_user_in_db(db_dir):
    '''
    query all users stored in current database
    '''
    users = []
    conn = sqlite3.connect(db_dir)
    db_cursor = conn.cursor()
    search_query = "SELECT * FROM user_db"
    search_data = db_cursor.execute(search_query)
    conn.commit()
    for i in search_data.fetchall():
        i = list(i)
        users.append(i[0])
    conn.close()
    return users

def insert_user_to_db(user, db_dir):
    '''
    insert user data to database
    '''
    conn = sqlite3.connect(db_dir)
    db_cursor = conn.cursor()
    insert_query = 'INSERT INTO user_db (userid) VALUES (?)'
    db_cursor.execute(insert_query, (user,))
    conn.commit()
    conn.close()

def search_user_duplicate(user, db_dir):
    '''
    searches user data in database
    returns true if there exists same user
    '''
    conn = sqlite3.connect(db_dir)
    db_cursor = conn.cursor()
    search_query = "SELECT userid FROM user_db WHERE userid=(?)"
    duplicate = db_cursor.execute(search_query, (user,))
    conn.commit()
    flag = len(duplicate.fetchall())>0
    conn.close()
    return flag

def parse_user(users, db_dir):
    '''
    it takes users and returns new chats that does not exist in database
    '''
    for user in users:
        user = str(user)
        if search_user_duplicate(user, db_dir) is True:
            continue
        insert_user_to_db(user, db_dir)
    return list(set(users))


In [3]:
'''
Functions related to web crawling and parsing
'''
from bs4 import BeautifulSoup
import requests
import telegram

def get_web_cookie(url, data):
    '''
    returns cookies from the url using data given as parameter
    '''
    cookies = ''
    with requests.Session() as web_session:
        web_session.post(url, data=data)
        cookies = dict(web_session.cookies)
    return cookies

def get_web_html(url, cookie):
    '''
    returns html text from the url using cookie given as parameter
    '''
    html = ''
    with requests.Session() as web_session:
        response = web_session.get(url, cookies = cookie)
        html = response.text
    return html

def make_web_recruitment_data_json(company, content, deadline):
    '''
    returns json that has company, content and deadline as keys
    '''
    return {
        'company': company,
        'content': content,
        'deadline': deadline
    }

def parse_web_recruitment_skku(html):
    '''
    returns recruitment data of SKKU
    '''
    recruitment_json = []
    soup = BeautifulSoup(html, "html.parser")
    rough_data = soup.findAll('table', {"class": "listTable1"})[0].findAll('tr')
    for i in range(1, len(rough_data)):
        company_and_content = rough_data[i].findAll('td', {"class": "tal"})
        company = company_and_content[0].text
        content = company_and_content[1].text
        deadline = rough_data[i].findAll('td', {"class": "fs_sm"})[0].text
        recruitment_json.append(make_web_recruitment_data_json(company, content, deadline))
    return recruitment_json

def parse_web_recruitment_jobkorea(html):
    '''
    returns recruitment data of JOBKOREA
    '''
    recruitment_json = []
    soup = BeautifulSoup(html, "html.parser")
    rough_data = soup.select('ol', {"class": "rankList"})[0].select('li')
    for i in rough_data:
        company = i.findAll('a', {"class": "coLink"})[0].select('b')[0].text
        content = i.findAll('a', {"class": "link"})[0].text
        deadline = i.findAll('span', {"class": "day"})[0].text
        recruitment_json.append(make_web_recruitment_data_json(company, content, deadline))
    return recruitment_json

def make_web_telegram_chat_data_json(chat_id, user_id, content):
    '''
    returns json list that has telegram chat
    '''
    return {
        'chat_id': chat_id,
        'user_id': user_id,
        'content': content
    }

def get_telegram_chat_of_last_24_hours(telegram_token):
    '''
    returns message sent to bot via telegram in last 24 hours
    '''
    bot = telegram.Bot(token=telegram_token)
    chat_json = []
    for i in bot.getUpdates(timeout=30):
        chat_id = str(i['message']['message_id'])
        user_id = str(i['message']['chat']['id'])
        content = i['message']['text']
        chat_json.append(make_web_telegram_chat_data_json(chat_id, user_id, content))
    return chat_json

def send_telegram_message(telegram_token, user_id, content):
    '''
    bot sends message given as argument to user
    '''
    bot = telegram.Bot(token = telegram_token)
    bot.send_message(chat_id=str(user_id), text=content)


In [4]:
'''
funtions not related to web and database
'''

def make_bot_message_form(recruitment):
    '''
    make message form that bot sends to users
    '''
    return "["+recruitment['company']+"] \n"+recruitment['content']+" \nDue: "+recruitment['deadline']


In [2]:
DB_DIR = "./user.db"

In [20]:
conn = sqlite3.connect(DB_DIR)
db_cursor = conn.cursor()
search_query = "SELECT * FROM rec_db"
duplicate = db_cursor.execute(search_query)
conn.commit()

db_list = duplicate.fetchall()
# for db in db_list:
#     print(db)
print(len(db_list))
# print(duplicate.fetchall())

conn.close()

3953
